# Classification using Naive Bayes

In [1]:
import pandas as pd

DF=pd.read_csv('../data/reviews_Video_Games_5-over10-data_frame.csv')
DF.head()

,text,utility,actual,anoth,back,bad,best,big,bit,buy,...,time,tri,turn,version,way,well,work,world,worth,year
0,'pay unlock content think instal game struggl ...,0,0.0,0.052,0.0,0.0,0.000,0.0,0.0,0.091,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
1,'awesom game crash frequent got version instea...,1,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.046,...,0.069,0.047,0.029,0.147,0.000,0.020,0.024,0.0,0.028,0.052
2,'step dirt terrif love play dirt thought graph...,1,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,...,0.000,0.000,0.000,0.000,0.000,0.049,0.000,0.0,0.000,0.000
3,'fun pretti fun game buy car track onlin store...,1,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.138,...,0.052,0.000,0.000,0.000,0.064,0.000,0.000,0.0,0.000,0.000
4,'best graphic game far must gamer realli hit h...,1,0.0,0.000,0.0,0.0,0.105,0.0,0.0,0.148,...,0.000,0.000,0.000,0.000,0.000,0.044,0.000,0.0,0.000,0.000


In [2]:
DF.tail()

,text,utility,actual,anoth,back,bad,best,big,bit,buy,...,time,tri,turn,version,way,well,work,world,worth,year
30126,'scam real bundl real mario kart bundl red box...,1,0.000,0.0,0.0,0.0,0.057,0.0,0.0,0.053,...,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
30127,'get bundl includ extra wheel control delux ma...,1,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,...,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
30128,'fake bundl packag red show steer wheel retail...,1,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,...,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
30129,'look like gouger get packag mine arriv box red',1,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,...,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
30130,'buy look european version real retail store t...,1,0.082,0.0,0.0,0.0,0.077,0.0,0.0,0.145,...,0.0,0.074,0.0,0.186,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import csv

data=[]
target=[]

reader=csv.reader(open('../data/reviews_Video_Games_5-over10-data_frame.csv','rt'))

next(reader)
for row in reader:
  target.append(row[1])
  l=[]
  for e in row[2:]:
    l.append(float(e))
  data.append(l)

In [4]:
from sklearn.naive_bayes import GaussianNB

gnb=GaussianNB()
pred=gnb.fit(data,target).predict(data)

In [5]:
tot=len(data)
corrected=(target == pred).sum()
uncorrected=(target != pred).sum()
print("Number of corrected points out of a total %d points : %d" % (tot,corrected))
print("Number of mislabeled points out of a total %d points : %d" % (tot,uncorrected))
print("Accuracy : %f" % (corrected/tot))

Number of corrected points out of a total 30131 points : 19581
Number of mislabeled points out of a total 30131 points : 10550
Accuracy : 0.649862


#Test

In [0]:
def compute_tfidf(doc,cv,tfidf_transformer,attributes):
  #generate tf-idf for the given document
  tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) 
  #sort the tf-idf vectors by descending order of scores
  sorted_items=sort_coo(tf_idf_vector.tocoo())
  #extract all items from document
  keywords=extract_topn_from_vector(feature_names,sorted_items,len(doc))
  dict_doc={}
  for e in attributes:
    if e in keywords:
      dict_doc[e]=keywords[e]
    else:
      dict_doc[e]=0
  return dict_doc

In [26]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
porter_stemmer = PorterStemmer()
en_stops = set(stopwords.words('english'))

#binary search to improve efficency
def binary_search(e,l,inizio,fine):
  if inizio>fine or e<l[inizio] or e>l[fine-1]:
    return False
  else:
    mezzo=int((inizio+fine)/2)
    m=l[mezzo]
    if e==m:
      return True
    elif e>m:
      return binary_search(e,l,mezzo,fine)
    else:
      return binary_search(e,l,inizio,mezzo)

#removing symbols and adding space after them
def prepreprocess(frase):
  s=''
  syms=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','è','é','à','ò','ì','ù']
  for char in frase:
    if char not in syms:
      s+=' '
    else:
      s+=char
  return s

#preprocess using prepreprocess fase and stemming and removing stopwords
def preprocess(frase,en_words):
  l=[]
  for word in prepreprocess(frase.lower()).split(' '):
    w=word.lower()
    if w not in en_stops and len(w)>2 and binary_search(w,en_words,0,len(en_words)):
        l.append(porter_stemmer.stem(w))
  return repr(' '.join(l))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
summary='Great game of all the time!!!'
reviewText="I've never played this game first, but since the initial install I've fallen in love <3 !!! It's amazing because shooting mode is very easy and level of entertainment is fantastic! I'll play for hour and hour so long!!! Very recommended!!!!"

# summary='hahah'
# reviewText="I like obama because he loves me so much and other people wanna play this game"

en_words=sorted(open('/content/drive/My Drive/uni/AmazonReviews/data/en_words.txt', 'r').read().lower().split('\n'))
doc=preprocess(str(summary+' '+reviewText),en_words)

from sklearn.feature_extraction.text import TfidfVectorizer

feature_names=cv.get_feature_names()

dict_doc=compute_tfidf(doc,cv,tfidf_transformer,list(DF.columns[2:]))

data_doc=[]
for v in dict_doc.values():
  data_doc.append(float(v))

NameError: ignored

In [0]:
import numpy as np

gnb_pf=GaussianNB()
gnb_pf.partial_fit(data,target,np.unique(target))
print(gnb_pf.predict([data_doc]))

['0']


Classification using Naive-Bayes and K-Fold cross validation

In [6]:
import csv
import numpy as np

data=[]
target=[]

reader=csv.reader(open('../data/reviews_Video_Games_5-over10-data_frame.csv','rt'))

next(reader)
for row in reader:
    target.append(int(row[1]))
    l=[]
    for e in row[2:]:
        l.append(np.float64(e))
    data.append(np.array(l))
    
data=np.array(data)
target=np.array(target)

In [7]:
from sklearn.model_selection import KFold

kf=KFold(n_splits=10, shuffle=True, random_state=None)
for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]

In [8]:
from sklearn.naive_bayes import GaussianNB

prediction=GaussianNB().fit(X_train, y_train).predict(X_test)

In [9]:
tp=0
tn=0
fp=0
fn=0

size=len(prediction)
for i in range (0,size-1):
    real=y_test[i]
    pred=prediction[i]
    if pred==1:
        if real==1:
            tp+=1
        else:
            fp+=1
    elif pred==0:
        if real==0:
            tn+=1
        else:
            fn+=1
            
print("true positive:",tp,"(",tp/size*100,"% )")
print("true negative:",tn,"(",tn/size*100,"% )")
print("false positive:",fp,"(",fp/size*100,"% )")
print("false negative:",fn,"(",fn/size*100,"% )")

print("\naccuracy:",(tp+tn)/size*100,"%")

true positive: 1212 ( 40.22568868237637 % )
true negative: 721 ( 23.929638234317956 % )
false positive: 692 ( 22.96714238300697 % )
false negative: 387 ( 12.844341188184533 % )

accuracy: 64.15532691669432 %


Classification using SVM

In [11]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

bankdata = pd.read_csv("../data/reviews_Video_Games_5-over10-data_frame.csv")
#bankdata.head()

In [12]:
#To divide the data into attributes and labels, execute the following code:
#In the first line of the script above, all the columns of the bankdata dataframe are being stored in 
#the X variable except the "Class" column, 
#which is the label column. The drop() method drops this column. In the second line, only the class 
#column is being stored in the y variable. 
#At this point of time X variable contains attributes while y variable contains corresponding labels.
X = bankdata.drop('utility', axis=1).drop('text', axis=1)
y = bankdata['utility']

In [13]:
#the model_selection library of the Scikit-Learn library contains the train_test_split method that allows us 
#to seamlessly divide data into training and test sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [14]:
#In the case of a simple SVM we simply set this parameter as "linear" since simple SVMs can only classify linearly separable data.
from sklearn.svm import SVC
import time

start = time.time()

svclassifier = SVC(kernel='rbf', gamma='auto') #Linear Kernel (linear); Polynomial K. (poly); Gaussian K. (rbf); Sigmoid K. (sigmoid)
svclassifier.fit(X_train, y_train)

end = time.time()
print("Time:",end-start,"seconds")

Time: 364.8301224708557 seconds


In [15]:
#To make predictions, the predict method of the SVC class is used.
import time

start=time.time()

y_pred = svclassifier.predict(X_test)

end=time.time()
print("Time:",end-start,"seconds")

Time: 91.02998352050781 seconds


In [41]:
#Evaluating the Algorithm Precision, Recall, F1 e confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[  24 2753]
 [   6 3244]]
              precision    recall  f1-score   support

           0       0.80      0.01      0.02      2777
           1       0.54      1.00      0.70      3250

    accuracy                           0.54      6027
   macro avg       0.67      0.50      0.36      6027
weighted avg       0.66      0.54      0.39      6027



# Classification using Neural Network

In [45]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import datasets

Using TensorFlow backend.


In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
# load dataset
dataframe = pd.read_csv('/content/drive/My Drive/uni/AmazonReviews/data/reviews_Video_Games_5-data_frame.csv')#, header=None)
dataset = dataframe.values
X = dataset[:,2:].astype(float)
Y = dataset[:,1]

In [43]:
X

array([[0.   , 0.058, 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.088, 0.   , 0.   , ..., 0.   , 0.   , 0.   ]])

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [0]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=100, activation='relu'))
	model.add(Dense(2, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [0]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [0]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [0]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

